In [17]:
import cv2
import numpy as np

In [19]:
def realtime_object_detection(img,video_device,min_match_number): 
    cap = cv2.VideoCapture(video_device)
    detector = cv2.SIFT_create()
    matching = cv2.BFMatcher()
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_kpts,img_desc = detector.detectAndCompute(img_gray,None)

    while cap.isOpened():
        ret,frame = cap.read()
        if ret :
            frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            query_kpts,query_desc = detector.detectAndCompute(frame_gray,None)
            match = matching.knnMatch(img_desc,query_desc,k = 2)
            goodmatch = list()
            for m,n in match :
                if m.distance < 0.7*n.distance :
                    goodmatch.append(m)
            if len(goodmatch) > min_match_number :
                src_pts = np.float32([ img_kpts[m.queryIdx].pt for m in goodmatch]).reshape(-1,1,2)
                dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in goodmatch]).reshape(-1,1,2)
                src_pts,dst_pts = np.float32((src_pts,dst_pts))
                H,status=cv2.findHomography(src_pts,dst_pts,cv2.RANSAC,3.0)
                h,w=img.shape[:2]
                imgBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
                queryBorder=cv2.perspectiveTransform(imgBorder,H)
                cv2.polylines(frame,[np.int32(queryBorder)],True,(0,255,0),5)
                cv2.imshow('result',frame)

            delay = cv2.waitKey(int(1000/30)) & 0xFF
            if delay == 24 or delay == ord('q') :
                break
            else :
                frame_gray = frame_gray.copy()
        else :
            break
    cap.release()
    cv2.destroyAllWindows()

tem_img = cv2.imread('Template-1.png',1)
realtime_object_detection(tem_img,'videos/stereo/left_output',20)

            